### Copilot Project Detail
Project - Co-Pilot Competition - Data Engineering Team <br/>
Author - Zubair Khan <br/>
Documentation of the project - https://eydti.atlassian.net/wiki/spaces/DART/pages/65853882424/Co-pilot+Competition+-+PNR+OD+Construction

Solution Documentation - https://eydti.atlassian.net/wiki/spaces/DART/pages/65853882424/Co-pilot+Competition+-+PNR+OD+Construction

##Importing of required libraries


In [0]:
import re
import math
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

## Data file paths

In [0]:
#These paths I gave as a input to Copilot
pnr_sample_path = "/mnt/ppeedp/raw/competition/pnr_sample"
distance_master_path = "/mnt/stppeedp/ppeedp/CBI2/production/reference_zone/distance_master"
location_master_path = "/mnt/stppeedp/ppeedp/raw/eag/ey/test_cbi_reference_data_loader/target_dir/Location_master"
backtrack_exception_master_path = "/mnt/stppeedp/ppeedp/raw/eag/ey/test_cbi_reference_data_loader/target_dir/BacktrackExceptionmaster"

# Generated copilot code to load delta table path ( prompt 1)


In [0]:
# Code from Copilot
pnr_df = spark.read.format("delta").load(pnr_sample_path)
distance_master_df = spark.read.format("delta").load(distance_master_path)
location_master_df = spark.read.format("delta").load(location_master_path)
backtrack_exception_master_df = spark.read.format("delta").load(backtrack_exception_master_path)

###Structure of pnr_df ( Copilot Prompt )

PNRHash:string, 
SEG_SEQ_NBR:string, 
ORIG_AP_CD:string, 
DESTN_AP_CD:string, 
OPT_ALN_CD:string, 
OPT_ALN_FLT_NBR:string, 
MKT_ALN_CD:string, 
fl_dt:string

###Structure of distance_master_df ( Copilot Prompt )

ID:string, 
airport_origin_code:string,
airport_destination_code:string,
distance_in_km:integer,
distance_in_miles:decimal(14,2),
source:string,
from_dt:date,
thru_dt:date

###Structure of location_master_df ( Copilot Prompt )

ap_cd_val:string,
latitude:string,
longitude:string

###Structure of backtrack_exception_master_df ( Copilot Prompt )
orig:string,
dest:string

##Join pnr_df with distance_master on (Copilot prompt 2)
 (ORIG_AP_CD = airport_origin_code and DESTN_AP_CD = airport_destination_code) or  (ORIG_AP_CD = airport_destination_code  and DESTN_AP_CD = airport_origin_code)

In [0]:
# Code from Copilot with given joining condition
Joined_df = pnr_df.join(distance_master_df,(
    ((col("ORIG_AP_CD") == col("airport_origin_code")) &(col("DESTN_AP_CD") == col("airport_destination_code"))) | ((col("ORIG_AP_CD") == col("airport_destination_code")) & (col("DESTN_AP_CD") == col("airport_origin_code"))) ),how ="left")

##Join the resultant dataframe with location_master on ORIG_AP_CD = ap_cd_val (Copilot Prompt 3) 

In [0]:
#Code given by Copilot after giving joining condition
Joined_df = Joined_df.join(location_master_df,col("ORIG_AP_CD") == col("ap_cd_val"),how ="left")

##Rename column ap_cd_val with ap_cd_val_origin, latitude with latitude_origin and longitude with longitude_origin (Copilot Prompt4)

In [0]:
# code from copilot to rename
newJoined_df=Joined_df.withColumnRenamed("ap_cd_val","ap_cd_val_origin") \
.withColumnRenamed("latitude","latitude_origin") \
.withColumnRenamed("longitude","longitude_origin")

##Join the new_Joined_df with location_master on DESTN_AP_CD = ap_cd_val (Prompt to Copilot 5) 

In [0]:
#Code given by Copilot after giving joining condition
newJoined_df=newJoined_df.join(location_master_df,col("DESTN_AP_CD") == col("ap_cd_val"),how ="left")

%md
##Rename the following columns (Copilot prompt 6)
ap_cd_val with ap_cd_val_desti, latitude with latitude_desti and longitude with longitude_desti, distance_in_km with distance_in_km_OD and column id with id_OD

In [0]:
# code from copilot to rename
#Renaming the columns to avoid confusion
Joined_df=newJoined_df.withColumnRenamed("ap_cd_val","ap_cd_val_desti") \
.withColumnRenamed("latitude","latitude_desti") \
.withColumnRenamed("longitude","longitude_desti") \
.withColumnRenamed("distance_in_km","distance_in_km_OD") \
.withColumnRenamed("id","id_OD")

#Create a Pyspark UDF to convert DMS to Decimal degress ( copilot prompt 7)
This udf should take one parameter DMS, it should calculates the decimal degrees based on degrees, minutes, seconds, direction values and returns the result. If the direction is either 'S' or 'W', it negates the result because these directions represent negative values in the coordinate system.

In [0]:
# code from copilot
# Define a UDF to convert DMS to DD
def dms_to_decimal(dms):
    """
    Converting degrees, minutes, and seconds to decimal degrees.

    :degrees: Degrees part of the angle.
    :minutes: Minutes part of the angle.
    :seconds: Seconds part of the angle.
    :direction: 'N', 'S', 'E', or 'W' for North, South, East, or West.
    :return: Decimal degrees.
    """
    if dms is None:
        return None
    match = re.match(r"(\d+).(\d+).(\d+)([NSWE])", dms)
    if match is None:
        return None
    degrees, minutes, seconds, direction = match.groups()
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ['S','W']:
        dd*= -1
    return dd

dms_to_decimal_udf = udf(dms_to_decimal, FloatType())

# Apply the UDF to all the latitude and longitude columns (Copilot prompt 8)

In [0]:

## Copilot code from the above prompt
Joined_df = Joined_df.withColumn("lat_origin", dms_to_decimal_udf(Joined_df["latitude_origin"]))
Joined_df = Joined_df.withColumn("lon_origin", dms_to_decimal_udf(Joined_df["longitude_origin"]))
Joined_df = Joined_df.withColumn("lat_desti", dms_to_decimal_udf(Joined_df["latitude_desti"]))
transformed_df = Joined_df.withColumn("lon_desti", dms_to_decimal_udf(Joined_df["longitude_desti"]))


# Write a Python UDF to calculate the distance between two points using their latitude and longitude. The UDF accept the latitude and longitude for both points and return the distance in kilometers using the Haversine formula ( copilot prompt 9)

Haversine formula:
dlon = lon2 - lon1
dlat = lat2 - lat1
a = sin^2(dlat/2) + cos(lat1) * cos(lat2) * sin^2(dlon/2)
c = 2 * atan2(sqrt(a), sqrt(1-a))
distance = R * c
where:
- lon1, lon2, lat1, and lat2 are in radians
- R is the radius of the Earth




In [0]:
# Copilot Code from the above prompt
def haversine_distance(lat1, lon1, lat2, lon2):
    # Converting latitude and longitude in radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = 6371 * c  # Earth's radius in kilometers

    return distance
    
haversine_udf = udf(haversine_distance, DoubleType())

In [0]:

# OD Distance calculation in_km using window function (Without Copilot)

window = Window.partitionBy('PNRHash').orderBy('SEG_SEQ_NBR')
transformed_df = transformed_df.withColumn("distance_in_km_OD",when(col("id_OD").isNull(), haversine_udf(col("lon_origin"), col("lat_origin"), col("lon_desti"), col("lat_desti"))).otherwise(col("distance_in_km_OD")))

transformed_df = transformed_df.withColumn('OD_total_distance', sum('distance_in_km_OD').over(Window.partitionBy('PNRHash')))

#Extarcting the list of exception for source and destination from circuit logic(Code added by myself)
exception_list = backtrack_exception_master_df.collect()

In [0]:
transformed_df_dist = transformed_df.withColumn("dist_between_src_dst",haversine_udf(col("lat_origin"),col("lon_origin"),col("lat_desti"),col("lon_desti")))
transformed_df_dist.display()

PNRHash,SEG_SEQ_NBR,ORIG_AP_CD,DESTN_AP_CD,OPT_ALN_CD,OPT_ALN_FLT_NBR,MKT_ALN_CD,fl_dt,id_OD,airport_origin_code,airport_destination_code,distance_in_km_OD,distance_in_miles,source,from_dt,thru_dt,ap_cd_val_origin,latitude_origin,longitude_origin,ap_cd_val_desti,latitude_desti,longitude_desti,lat_origin,lon_origin,lat_desti,lon_desti,OD_total_distance,dist_between_src_dst
00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec,1,BLR,AUH,EY,239,EY,2/25/2024,AUHBLR,AUH,BLR,2724.0,1702.50,CBI,2015-01-01,9999-12-31,BLR,13.11.56N,077.42.20E,AUH,24.25.59N,054.39.04E,13.198889,77.70556,24.433056,54.65111,8169.0,2722.9795131668275
00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec,2,AUH,FCO,EY,85,EY,2/25/2024,AUHFCO,AUH,FCO,4344.0,2715.00,CBI,2015-01-01,9999-12-31,AUH,24.25.59N,054.39.04E,FCO,41.48.01N,012.14.20E,24.433056,54.65111,41.800278,12.238889,8169.0,4340.102470034667
00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec,3,FCO,CDG,AZ,316,EY,2/25/2024,CDGFCO,CDG,FCO,1101.0,688.13,CBI,2015-01-01,9999-12-31,FCO,41.48.01N,012.14.20E,CDG,49.00.35N,002.32.52E,41.800278,12.238889,49.009724,2.547778,8169.0,1100.5786289026207
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,1,MUC,AUH,EY,6,EY,2/25/2024,AUHMUC,AUH,MUC,4584.0,2865.00,CBI,2015-01-01,9999-12-31,MUC,48.21.14N,011.47.10E,AUH,24.25.59N,054.39.04E,48.35389,11.786111,24.433056,54.65111,16064.0,4579.999538996004
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,2,AUH,CCU,EY,256,EY,2/25/2024,AUHCCU,AUH,CCU,3448.0,2155.00,CBI,2015-01-01,9999-12-31,AUH,24.25.59N,054.39.04E,CCU,22.39.17N,088.26.48E,24.433056,54.65111,22.654722,88.44666,16064.0,3442.328254533196
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,3,CCU,AUH,EY,257,EY,2/25/2024,AUHCCU,AUH,CCU,3448.0,2155.00,CBI,2015-01-01,9999-12-31,CCU,22.39.17N,088.26.48E,AUH,24.25.59N,054.39.04E,22.654722,88.44666,24.433056,54.65111,16064.0,3442.328254533196
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,4,AUH,MUC,EY,5,EY,2/25/2024,AUHMUC,AUH,MUC,4584.0,2865.00,CBI,2015-01-01,9999-12-31,AUH,24.25.59N,054.39.04E,MUC,48.21.14N,011.47.10E,24.433056,54.65111,48.35389,11.786111,16064.0,4579.999538996004
06bba1f7e357f9f338d696fd45b6c18cd5e0c26633df47c56e4bbfd2c21485d9,1,LHR,AUH,EY,20,EY,2/25/2024,AUHLHR,AUH,LHR,5525.0,3453.13,CBI,2015-01-01,9999-12-31,LHR,51.28.39N,000.27.41W,AUH,24.25.59N,054.39.04E,51.4775,-0.4613889,24.433056,54.65111,14996.0,5516.773347110172
06bba1f7e357f9f338d696fd45b6c18cd5e0c26633df47c56e4bbfd2c21485d9,2,AUH,BOM,EY,196,EY,2/25/2024,AUHBOM,AUH,BOM,1973.0,1233.13,CBI,2015-01-01,9999-12-31,AUH,24.25.59N,054.39.04E,BOM,19.05.19N,072.52.05E,24.433056,54.65111,19.088612,72.86806,14996.0,1970.9031512314768
06bba1f7e357f9f338d696fd45b6c18cd5e0c26633df47c56e4bbfd2c21485d9,3,BOM,AUH,EY,197,EY,2/25/2024,AUHBOM,AUH,BOM,1973.0,1233.13,CBI,2015-01-01,9999-12-31,BOM,19.05.19N,072.52.05E,AUH,24.25.59N,054.39.04E,19.088612,72.86806,24.433056,54.65111,14996.0,1970.9031512314768


In [0]:


rawdf=transformed_df_dist.withColumn("id",concat(col('PNRHash'),lit("|"),col('SEG_SEQ_NBR')))
rawdf=rawdf.withColumn("org",concat(col('ORIG_AP_CD'),lit(" "),col('OPT_ALN_CD'),lit(" "),col('DESTN_AP_CD'),lit(" ")))
rawdf=rawdf.withColumn("org1",concat(col('ORIG_AP_CD'),lit("-"),col('DESTN_AP_CD')))

df_raw1=rawdf.groupby('PNRHash').agg(concat_ws('|', expr('collect_list(org)')).alias('final'),concat_ws('|', expr('collect_list(org1)')).alias('final2'))

substring_to_replace =r'(\w{3})\s*\|'
replacement_substring =r''
df_replaced = df_raw1.withColumn("journey_operating", regexp_replace("final",substring_to_replace,replacement_substring)).withColumn("online_od_itinerary",regexp_replace("final2",substring_to_replace,replacement_substring))

df_transformed=rawdf.join(df_replaced,df_replaced.PNRHash==rawdf.PNRHash).select(rawdf['*'],df_replaced.journey_operating,df_replaced.online_od_itinerary).drop("org","org1").withColumn("split_str",split(trim(col("journey_operating"))," ")).withColumn("point_of_commencement",col("split_str") [0]).withColumn("point_of_finish",col("split_str") [size(col("split_str"))-1]).drop("split_str")
df_transformed.display()


PNRHash,SEG_SEQ_NBR,ORIG_AP_CD,DESTN_AP_CD,OPT_ALN_CD,OPT_ALN_FLT_NBR,MKT_ALN_CD,fl_dt,id_OD,airport_origin_code,airport_destination_code,distance_in_km_OD,distance_in_miles,source,from_dt,thru_dt,ap_cd_val_origin,latitude_origin,longitude_origin,ap_cd_val_desti,latitude_desti,longitude_desti,lat_origin,lon_origin,lat_desti,lon_desti,OD_total_distance,dist_between_src_dst,id,journey_operating,online_od_itinerary,point_of_commencement,point_of_finish
00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec,1,BLR,AUH,EY,239,EY,2/25/2024,AUHBLR,AUH,BLR,2724.0,1702.50,CBI,2015-01-01,9999-12-31,BLR,13.11.56N,077.42.20E,AUH,24.25.59N,054.39.04E,13.198889,77.70556,24.433056,54.65111,8169.0,2722.9795131668275,00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec|1,BLR EY AUH EY FCO AZ CDG,BLR-AUH-FCO-CDG,BLR,CDG
00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec,2,AUH,FCO,EY,85,EY,2/25/2024,AUHFCO,AUH,FCO,4344.0,2715.00,CBI,2015-01-01,9999-12-31,AUH,24.25.59N,054.39.04E,FCO,41.48.01N,012.14.20E,24.433056,54.65111,41.800278,12.238889,8169.0,4340.102470034667,00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec|2,BLR EY AUH EY FCO AZ CDG,BLR-AUH-FCO-CDG,BLR,CDG
00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec,3,FCO,CDG,AZ,316,EY,2/25/2024,CDGFCO,CDG,FCO,1101.0,688.13,CBI,2015-01-01,9999-12-31,FCO,41.48.01N,012.14.20E,CDG,49.00.35N,002.32.52E,41.800278,12.238889,49.009724,2.547778,8169.0,1100.5786289026207,00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec|3,BLR EY AUH EY FCO AZ CDG,BLR-AUH-FCO-CDG,BLR,CDG
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,1,MUC,AUH,EY,6,EY,2/25/2024,AUHMUC,AUH,MUC,4584.0,2865.00,CBI,2015-01-01,9999-12-31,MUC,48.21.14N,011.47.10E,AUH,24.25.59N,054.39.04E,48.35389,11.786111,24.433056,54.65111,16064.0,4579.999538996004,04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf|1,MUC EY AUH EY CCU EY AUH EY MUC,MUC-AUH-CCU-AUH-MUC,MUC,MUC
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,2,AUH,CCU,EY,256,EY,2/25/2024,AUHCCU,AUH,CCU,3448.0,2155.00,CBI,2015-01-01,9999-12-31,AUH,24.25.59N,054.39.04E,CCU,22.39.17N,088.26.48E,24.433056,54.65111,22.654722,88.44666,16064.0,3442.328254533196,04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf|2,MUC EY AUH EY CCU EY AUH EY MUC,MUC-AUH-CCU-AUH-MUC,MUC,MUC
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,3,CCU,AUH,EY,257,EY,2/25/2024,AUHCCU,AUH,CCU,3448.0,2155.00,CBI,2015-01-01,9999-12-31,CCU,22.39.17N,088.26.48E,AUH,24.25.59N,054.39.04E,22.654722,88.44666,24.433056,54.65111,16064.0,3442.328254533196,04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf|3,MUC EY AUH EY CCU EY AUH EY MUC,MUC-AUH-CCU-AUH-MUC,MUC,MUC
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,4,AUH,MUC,EY,5,EY,2/25/2024,AUHMUC,AUH,MUC,4584.0,2865.00,CBI,2015-01-01,9999-12-31,AUH,24.25.59N,054.39.04E,MUC,48.21.14N,011.47.10E,24.433056,54.65111,48.35389,11.786111,16064.0,4579.999538996004,04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf|4,MUC EY AUH EY CCU EY AUH EY MUC,MUC-AUH-CCU-AUH-MUC,MUC,MUC
06bba1f7e357f9f338d696fd45b6c18cd5e0c26633df47c56e4bbfd2c21485d9,1,LHR,AUH,EY,20,EY,2/25/2024,AUHLHR,AUH,LHR,5525.0,3453.13,CBI,2015-01-01,9999-12-31,LHR,51.28.39N,000.27.41W,AUH,24.25.59N,054.39.04E,51.4775,-0.4613889,24.433056,54.65111,14996.0,5516.773347110172,06bba1f7e357f9f338d696fd45b6c18cd5e0c26633df47c56e4bbfd2c21485d9|1,LHR EY AUH EY BOM EY AUH EY LHR,LHR-AUH-BOM-AUH-LHR,LHR,LHR
06bba1f7e357f9f338d696fd45b6c18cd5e0c26633df47c56e4bbfd2c21485d9,2,AUH,BOM,EY,196,EY,2/25/2024,AUHBOM,AUH,BOM,1973.0,1233.13,CBI,2015-01-01,9999-12-31,AUH,24.25.59N,054.39.04E,BOM,19.05.19N,072.52.05E,24.433056,54.65111,19.088612,72.86806,14996.0,1970.9031512314768,06bba1f7e357f9f338d696fd45b6c18cd5e0c26633df47c56e4bbfd2c21485d9|2,LHR EY AUH EY BOM EY AUH EY LHR,LHR-AUH-BOM-AUH-LHR,LHR,LHR
06bba1f7e357f9f338d696fd45b6c18cd5e0c26633df47c56e4bbfd2

In [0]:
from pyspark.sql.functions import split

def change_path(col):
    col_list = []
    l = col.split("-")  # Use split function instead of split method
    if len(l)<=2:
        return [[str(l[0]),str(l[1])]]
    else:
        for i in range(0, len(l)-1):  # Iterate until len(l)-1 to avoid index out of range
            if len(l[i:i+3]) == 3:  # Change the condition to check for 2 elements
                col_list.append(l[i:i+3])
        return col_list


change_path_udf = udf(change_path)

In [0]:
dict_c = spark.read.format("delta").load("/mnt/stppeedp/ppeedp/raw/eag/ey/test_cbi_reference_data_loader/target_dir/Location_master").rdd.collect()
dict_c_dic = {}
for i in dict_c:
    dict_c_dic[str(i.ap_cd_val)] =[ str(i.latitude),str(i.longitude)]
# print(dict_c_dic)
broadcast_dict_c_dic = spark.sparkContext.broadcast(dict_c_dic)

# applying circuit break on total_distance

In [0]:

def is_circuit_break(distance, total_distance, origin, desti):
    # Checking if thee given origin and destination exists in the exception list(Code added by myself)
    if any(x.orig == origin and x.dest == desti for x in exception_list):
        return False
    else:
        displacement_circuit = distance * 1.6
        if displacement_circuit < total_distance:
            return True
        else:
            return False

is_circuit_break_udf = udf(is_circuit_break, BooleanType())


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import ArrayType, StringType

df_5 = df_transformed.withColumn("ods_split",change_path_udf(col("online_od_itinerary")))



In [0]:
# Applying the UDF
Joined_df_1 = df_5.withColumn('is_circuit_break', is_circuit_break_udf(col('distance_in_km_OD'), col('OD_total_distance'),col('point_of_commencement'),col('point_of_finish')))

In [0]:
Joined_df_2 = Joined_df_1.withColumn('TRUE_OD', 
                   when(Joined_df_1['is_circuit_break'] == False, concat(Joined_df_1['point_of_commencement'],  Joined_df_1['point_of_finish']))
                   .otherwise(concat(Joined_df_1['ORIG_AP_CD'],  Joined_df_1['DESTN_AP_CD'])))

Joined_df_3 = Joined_df_2.withColumn('Online_OD', 
                   when((Joined_df_1['is_circuit_break'] == False) & (Joined_df_1['OPT_ALN_CD'] == 'EY'), concat(Joined_df_1['point_of_commencement'], lit('-'), Joined_df_1['DESTN_AP_CD']))
                   .otherwise(when(Joined_df_1['OPT_ALN_CD'] == 'EY', concat(Joined_df_1['ORIG_AP_CD'], lit('-'), Joined_df_1['DESTN_AP_CD']))))
Joined_df_3.display()

PNRHash,SEG_SEQ_NBR,ORIG_AP_CD,DESTN_AP_CD,OPT_ALN_CD,OPT_ALN_FLT_NBR,MKT_ALN_CD,fl_dt,id_OD,airport_origin_code,airport_destination_code,distance_in_km_OD,distance_in_miles,source,from_dt,thru_dt,ap_cd_val_origin,latitude_origin,longitude_origin,ap_cd_val_desti,latitude_desti,longitude_desti,lat_origin,lon_origin,lat_desti,lon_desti,OD_total_distance,dist_between_src_dst,id,journey_operating,online_od_itinerary,point_of_commencement,point_of_finish,ods_split,is_circuit_break,TRUE_OD,Online_OD
00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec,1,BLR,AUH,EY,239,EY,2/25/2024,AUHBLR,AUH,BLR,2724.0,1702.50,CBI,2015-01-01,9999-12-31,BLR,13.11.56N,077.42.20E,AUH,24.25.59N,054.39.04E,13.198889,77.70556,24.433056,54.65111,8169.0,2722.9795131668275,00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec|1,BLR EY AUH EY FCO AZ CDG,BLR-AUH-FCO-CDG,BLR,CDG,"[[BLR, AUH, FCO], [AUH, FCO, CDG]]",true,BLRAUH,BLR-AUH
00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec,2,AUH,FCO,EY,85,EY,2/25/2024,AUHFCO,AUH,FCO,4344.0,2715.00,CBI,2015-01-01,9999-12-31,AUH,24.25.59N,054.39.04E,FCO,41.48.01N,012.14.20E,24.433056,54.65111,41.800278,12.238889,8169.0,4340.102470034667,00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec|2,BLR EY AUH EY FCO AZ CDG,BLR-AUH-FCO-CDG,BLR,CDG,"[[BLR, AUH, FCO], [AUH, FCO, CDG]]",true,AUHFCO,AUH-FCO
00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec,3,FCO,CDG,AZ,316,EY,2/25/2024,CDGFCO,CDG,FCO,1101.0,688.13,CBI,2015-01-01,9999-12-31,FCO,41.48.01N,012.14.20E,CDG,49.00.35N,002.32.52E,41.800278,12.238889,49.009724,2.547778,8169.0,1100.5786289026207,00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec|3,BLR EY AUH EY FCO AZ CDG,BLR-AUH-FCO-CDG,BLR,CDG,"[[BLR, AUH, FCO], [AUH, FCO, CDG]]",true,FCOCDG,null
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,1,MUC,AUH,EY,6,EY,2/25/2024,AUHMUC,AUH,MUC,4584.0,2865.00,CBI,2015-01-01,9999-12-31,MUC,48.21.14N,011.47.10E,AUH,24.25.59N,054.39.04E,48.35389,11.786111,24.433056,54.65111,16064.0,4579.999538996004,04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf|1,MUC EY AUH EY CCU EY AUH EY MUC,MUC-AUH-CCU-AUH-MUC,MUC,MUC,"[[MUC, AUH, CCU], [AUH, CCU, AUH], [CCU, AUH, MUC]]",true,MUCAUH,MUC-AUH
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,2,AUH,CCU,EY,256,EY,2/25/2024,AUHCCU,AUH,CCU,3448.0,2155.00,CBI,2015-01-01,9999-12-31,AUH,24.25.59N,054.39.04E,CCU,22.39.17N,088.26.48E,24.433056,54.65111,22.654722,88.44666,16064.0,3442.328254533196,04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf|2,MUC EY AUH EY CCU EY AUH EY MUC,MUC-AUH-CCU-AUH-MUC,MUC,MUC,"[[MUC, AUH, CCU], [AUH, CCU, AUH], [CCU, AUH, MUC]]",true,AUHCCU,AUH-CCU
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,3,CCU,AUH,EY,257,EY,2/25/2024,AUHCCU,AUH,CCU,3448.0,2155.00,CBI,2015-01-01,9999-12-31,CCU,22.39.17N,088.26.48E,AUH,24.25.59N,054.39.04E,22.654722,88.44666,24.433056,54.65111,16064.0,3442.328254533196,04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf|3,MUC EY AUH EY CCU EY AUH EY MUC,MUC-AUH-CCU-AUH-MUC,MUC,MUC,"[[MUC, AUH, CCU], [AUH, CCU, AUH], [CCU, AUH, MUC]]",true,CCUAUH,CCU-AUH
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,4,AUH,MUC,EY,5,EY,2/25/2024,AUHMUC,AUH,MUC,4584.0,2865.00,CBI,2015-01-01,9999-12-31,AUH,24.25.59N,054.39.04E,MUC,48.21.14N,011.47.10E,24.433056,54.65111,48.35389,11.786111,16064.0,4579.999538996004,04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf|4,MUC EY AUH EY CCU EY AUH EY MUC,MUC-AUH-CCU-AUH-MUC,MUC,MUC,"[[MUC, AUH, CCU], [AUH, CCU, AUH], [CCU, AUH, MUC]]",true,AUHMUC,AUH-MUC
06bba1f7e357f9f338d696fd45b6c18cd5e0c26633df47c56e4bbfd2c21485d9,1,LHR,AUH,EY,20,EY,2/25/2024,AUHLHR,AUH,LHR,5525.0,3453.13,CBI,2015-01-01,9999-12-31,LHR,51.28.39N,000.27.41W,AUH,24.25.59N,054.39.04E,51.4775,-0.4613889,24.433056,54.65111,14996.0,5516.773347110172,06bba1f7e357f9f338d696fd45b6c18cd5e0c26633df47c56e4bbfd2c21485d9|1,

In [0]:
# code from copilot
Final_result = Joined_df_3.select([
    col("PNRHash").alias("id"),
    lit('01-01-2024').alias("PNRCreateDate"),
    col("OPT_ALN_CD").alias("IssueAirlineCode"),
    lit("XXXXXX").alias("PNR"),
    col("SEG_SEQ_NBR").alias("SegSequenceNumber"),
    col("TRUE_OD").alias("true_od"),
    col("Online_OD").alias("online_od"),
    col("online_od_itinerary").alias("online_od_itinerary"),
    col("fl_dt").alias("dep_date"),
    lit('00:00:00').alias("dep_time"),
    lit('01-01-9999').alias("arrival_date"),
    lit('00:00:00').alias("arrival_time"),
    col("OD_total_distance").alias("online_od_distance"),
    lit("999").alias("length_of_stay"),
    col("journey_operating").alias("journey_operating"),
    col("point_of_commencement"),
    col("point_of_finish"),
    col("OD_total_distance").alias("true_od_distance")
])

display(Final_result)

id,PNRCreateDate,IssueAirlineCode,PNR,SegSequenceNumber,true_od,online_od,online_od_itinerary,dep_date,dep_time,arrival_date,arrival_time,online_od_distance,length_of_stay,journey_operating,point_of_commencement,point_of_finish,true_od_distance
00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec,01-01-2024,EY,XXXXXX,1,BLRAUH,BLR-AUH,BLR-AUH-FCO-CDG,2/25/2024,00:00:00,01-01-9999,00:00:00,8169.0,999,BLR EY AUH EY FCO AZ CDG,BLR,CDG,8169.0
00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec,01-01-2024,EY,XXXXXX,2,AUHFCO,AUH-FCO,BLR-AUH-FCO-CDG,2/25/2024,00:00:00,01-01-9999,00:00:00,8169.0,999,BLR EY AUH EY FCO AZ CDG,BLR,CDG,8169.0
00fabe64dd540a0d55c71731442801b37518246a5105966447497523e75f6aec,01-01-2024,AZ,XXXXXX,3,FCOCDG,null,BLR-AUH-FCO-CDG,2/25/2024,00:00:00,01-01-9999,00:00:00,8169.0,999,BLR EY AUH EY FCO AZ CDG,BLR,CDG,8169.0
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,01-01-2024,EY,XXXXXX,1,MUCAUH,MUC-AUH,MUC-AUH-CCU-AUH-MUC,2/25/2024,00:00:00,01-01-9999,00:00:00,16064.0,999,MUC EY AUH EY CCU EY AUH EY MUC,MUC,MUC,16064.0
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,01-01-2024,EY,XXXXXX,2,AUHCCU,AUH-CCU,MUC-AUH-CCU-AUH-MUC,2/25/2024,00:00:00,01-01-9999,00:00:00,16064.0,999,MUC EY AUH EY CCU EY AUH EY MUC,MUC,MUC,16064.0
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,01-01-2024,EY,XXXXXX,3,CCUAUH,CCU-AUH,MUC-AUH-CCU-AUH-MUC,2/25/2024,00:00:00,01-01-9999,00:00:00,16064.0,999,MUC EY AUH EY CCU EY AUH EY MUC,MUC,MUC,16064.0
04debc19f6c744c77b869914c2e6717ce4e7194aa0eb947c2d914f6e7638a4cf,01-01-2024,EY,XXXXXX,4,AUHMUC,AUH-MUC,MUC-AUH-CCU-AUH-MUC,2/25/2024,00:00:00,01-01-9999,00:00:00,16064.0,999,MUC EY AUH EY CCU EY AUH EY MUC,MUC,MUC,16064.0
06bba1f7e357f9f338d696fd45b6c18cd5e0c26633df47c56e4bbfd2c21485d9,01-01-2024,EY,XXXXXX,1,LHRAUH,LHR-AUH,LHR-AUH-BOM-AUH-LHR,2/25/2024,00:00:00,01-01-9999,00:00:00,14996.0,999,LHR EY AUH EY BOM EY AUH EY LHR,LHR,LHR,14996.0
06bba1f7e357f9f338d696fd45b6c18cd5e0c26633df47c56e4bbfd2c21485d9,01-01-2024,EY,XXXXXX,2,AUHBOM,AUH-BOM,LHR-AUH-BOM-AUH-LHR,2/25/2024,00:00:00,01-01-9999,00:00:00,14996.0,999,LHR EY AUH EY BOM EY AUH EY LHR,LHR,LHR,14996.0
06bba1f7e357f9f338d696fd45b6c18cd5e0c26633df47c56e4bbfd2c21485d9,01-01-2024,EY,XXXXXX,3,BOMAUH,BOM-AUH,LHR-AUH-BOM-AUH-LHR,2/25/2024,00:00:00,01-01-9999,00:00:00,14996.0,999,LHR EY AUH EY BOM EY AUH EY LHR,LHR,LHR,14996.0
